In [13]:
import numpy as np
import cv2

#specify
CAR_PATH = ''
LP_PATH = ''

#read car in grayscale, color and threshold and LP in grayscale
car = cv2.imread(CAR_PATH,0)
lp = cv2.imread(LP_PATH,0)


# 16 different template sizes - greater chance of success


def lp_finder_16(car, lp):
    
    '''Function which returns list of  16 grayscale licence plate images, cropped from image of a car.
    Uses template matching, with 16 different template LP shapes.
    
    Params:
    car - grayscale image of a car
    lp - grayscale image of template licence plate'''
    
    #reshape car image to 600x400 - not used here, original images give better results if not huge
    #reshaping changes length and height ratios, which decreases algorithm efficiency 
    #since LP template shapes are hardcoded
    car_r = cv2.resize(car, (600,400))
    #thresholding car image
    car_th = cv2.threshold(car, 0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    
    #for 16 different LP sizes
    #getting resizing values for test licence plate
    shape1 = np.arange(150,230,5)
    shape2 = np.arange(56,84,1.5)
    #shape2 contains floats, converting to ints
    shape3 = []
    for i in shape2:
        shape3.append(int(i))
    #creating tuples (shape1,shape3) in single list
    sizes = zip(shape1,shape3)
    sizes = list(sizes)
    
    #getting threshold of LP, with otsu
    lp_th = cv2.threshold(lp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    # list of thresholded plates in all sizes
    plates_th = []
    for i in range(0, len(sizes)):
        plates_th.append(cv2.resize(lp_th, sizes[i]))

    #matching car with each LP size
    matches = []
    #list of matches for each size
    for i in plates_th:
        matches.append(cv2.matchTemplate(car_th, i, cv2.TM_CCOEFF))
    
    #appends location of matched squares to list, inverted to (x,y), for thresh
    #minMaxLoc finds maximum for each match
    locs = []
    for i in range(0, len(matches)):
        locs.append(tuple(cv2.minMaxLoc(matches[i])[3]))
    
    
    #creates masks for images, where only LP remains
    masks = []
    for i in range(0,len(plates_th)):
        black = np.zeros(car.shape, dtype=np.uint8)
        black = cv2.rectangle(black, locs[i], (locs[i][0]+plates_th[i].shape[1], locs[i][1]+plates_th[i].shape[0]), 255, -1)
        masks.append(black)
    
    #keeping only LP - cropped list contains all cropped images of LPs
    cropped = []
    for i in range(0,len(plates_th)):
        masked = cv2.bitwise_and(car, car, mask=masks[i])
        x,y = np.where(masked!=0)
        topx = x.min()
        bottomx = x.max()
        topy=y.min()
        bottomy=y.max()
        crop = masked[topx:bottomx,topy:bottomy]
        cropped.append(crop)
        
    return(cropped)   



# 4 different template sizes, to speed things up
# sizes taken as approximately smallest, biggest and 2 values in between from above 16 values
 
def lp_finder_4(car, lp):
    
    '''Function which returns list of 4 grayscale licence plate images, cropped from image of a car.
    Uses template matching, with 4 different template LP shapes.
    
    Params:
    car - grayscale image of a car
    lp - grayscale image of template licence plate'''
    
    #reshape car image to 600x400 - not used here, original images give better results if not huge
    #reshaping changes length and height ratios, which decreases algorithm efficiency 
    #since LP template shapes are hardcoded
    
    car_r = cv2.resize(car, (600,400))
    #thresholding car image
    car_th = cv2.threshold(car, 0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    
    #for 16 different LP sizes
    #getting resizing values for test licence plate
    shape1 = np.arange(150,230,25)
    shape2 = np.arange(56,84,9)
    #creating tuples (shape1,shape3) in single list
    sizes = zip(shape1,shape2)
    sizes = list(sizes)
    
    #getting threshold of LP, with otsu
    lp_th = cv2.threshold(lp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    # list of thresholded plates in all sizes
    plates_th = []
    for i in range(0, len(sizes)):
        plates_th.append(cv2.resize(lp_th, sizes[i]))

    #matching car with each LP size
    matches = []
    #list of matches for each size
    for i in plates_th:
        matches.append(cv2.matchTemplate(car_th, i, cv2.TM_CCOEFF))
    
    #appends location of matched squares to list, inverted to (x,y), for thresh
    #minMaxLoc finds maximum for each match
    locs = []
    for i in range(0, len(matches)):
        locs.append(tuple(cv2.minMaxLoc(matches[i])[3]))
    
    
    #creates masks for images, where only LP remains
    masks = []
    for i in range(0,len(plates_th)):
        black = np.zeros(car.shape, dtype=np.uint8)
        black = cv2.rectangle(black, locs[i], (locs[i][0]+plates_th[i].shape[1], locs[i][1]+plates_th[i].shape[0]), 255, -1)
        masks.append(black)
    
    #keeping only LP - cropped list contains all cropped images of LPs
    cropped = []
    for i in range(0,len(plates_th)):
        masked = cv2.bitwise_and(car, car, mask=masks[i])
        x,y = np.where(masked!=0)
        topx = x.min()
        bottomx = x.max()
        topy=y.min()
        bottomy=y.max()
        crop = masked[topx:bottomx,topy:bottomy]
        cropped.append(crop)
        
    return(cropped)